In [1]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import dash_bootstrap_components as dbc

# Read the Excel files
df = pd.read_excel("staff.xlsx")
df_types = pd.read_excel("types.xlsx")
df_departments = pd.read_excel("departments.xlsx")

# Create dropdown menu options from unique project names
project_options = [{'label': project, 'value': project} for project in df['Project'].unique()]

# Create dropdown menu options for project types
type_options = [{'label': project_type, 'value': project_type} for project_type in df_types['Type'].unique()]

# Filter out null values and create dropdown menu options from unique departments
department_options = [{'label': department, 'value': department} for department in df_departments['Department'].dropna().unique()]

# Add "Other" option for null department
department_options.append({'label': 'Other', 'value': 'Other'})

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define the layout of the app
app.layout = html.Div([
    html.H1("Deployment Plan"),
    html.Div([
        html.Label('Type of Project:'),
        dcc.Dropdown(
            id='type-dropdown',
            options=type_options,
            value=type_options[0]['value'] if type_options else None  # Default value
        )
    ]),
    html.Div([
        html.Label('Select Project:'),
        dcc.Dropdown(
            id='project-dropdown',
            options=project_options,
            value=project_options[0]['value']  # Default value
        ),
    ]),
    html.Div([
        html.Label('Select Department:'),
        dcc.Dropdown(
            id='department-dropdown',
            options=department_options,
            value=department_options[0]['value']  # Default value
        ),
    ]),
    dcc.Dropdown(
        id='job-dropdown',
        multi=True,
    ),
    dcc.Graph(id='deployment-plan'),
    html.Div(id='selected-project-type')  # Show the selected project type
])

# Define callback to update job dropdown options based on selected project and department
@app.callback(
    Output('job-dropdown', 'options'),
    [Input('project-dropdown', 'value'),
     Input('department-dropdown', 'value')]
)
def update_job_dropdown(selected_project, selected_department):
    # Filter the DataFrame based on the selected project and department
    filtered_jobs = df[(df['Project'] == selected_project) & (df_departments['Department'] == selected_department)]['Job'].unique()
    job_options = [{'label': job, 'value': job} for job in filtered_jobs]
    return job_options


if __name__ == '__main__':
    app.run_server(debug=True)
